## LIBRERIAS

In [1]:
import pandas as pd
import sqlite3

## INSTANCIAR A LA BASE DE DATOS

In [2]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("data/Bar_TB.db")

crsr = connection.cursor()

## FUNCIÓN PARA INTERACTUAR CON LA BASE DE DATOS

In [3]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)


## CREAR LAS TABLAS

Primero hay que verificar si ya están creadas, para que luego no de error. 

In [12]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0]) 

cerveza
bares
empleados
reparto


Ejecutando eso, me dice que no hay nada creado porque obviamente la base de datos está vacía ya que el archivo lo cree yo. 

Luego de creada la base de datos, si aparecen los nombres de las tablas: <br>
cerveza  <br>
bares <br>
empleados <br>
reparto. 

### TABLA CERVEZA

In [5]:
query_cerveza = """

CREATE TABLE cerveza (
    CodC VARCHAR (3),
    Envase VARCHAR (50),
    Capacidad FLOAT,
    Stock INT
)

"""

crsr.execute(query_cerveza)

In [11]:
pd.read_sql('SELECT * from cerveza', connection)

,CodC,Envase,Capacidad,Stock


### TABLA BARES

In [7]:
query_bares = """

CREATE TABLE bares (
    CodB VARCHAR (4),
    Nombre VARCHAR (50),
    Cif VARCHAR (10),
    Localidad VARCHAR (50)
)

"""

crsr.execute(query_bares)

### TABLA EMPLEADOS

In [9]:
query_empleados = """

CREATE TABLE empleados (
    CodE VARCHAR (2),
    Nombre VARCHAR (50),
    Sueldo INT
)

"""

crsr.execute(query_empleados)

### TABLA REPARTO

In [10]:
query_reparto = """

CREATE TABLE reparto (
    CodE VARCHAR (2),
    CodB VARCHAR (4),
    CodC VARCHAR (3),
    Fecha DATE,
    Cantidad INT
)

"""

crsr.execute(query_reparto)

## INSERTAR DATOS

In [16]:
# Insertamos datos en la TABLA CERVEZA

query_insert = """
INSERT INTO cerveza VALUES
('01', 'Botella', 0.2 , 3600),
('02', 'Botella', 0.33, 1200),
('03', 'Lata', 0.33, 2400),
('04', 'Botella', 1, 288),
('05', 'Barril', 60, 30)
"""

crsr.execute(query_insert)

In [17]:
pd.read_sql('SELECT * from cerveza', connection)

,CodC,Envase,Capacidad,Stock
0,01,Botella,0.20,3600
1,02,Botella,0.33,1200
2,03,Lata,0.33,2400
3,04,Botella,1.00,288
4,05,Barril,60.00,30


In [19]:
# Insertamos datos en la TABLA REPARTO

query_insert2 = """
INSERT INTO reparto (CodE, CodB, CodC, Fecha, Cantidad) VALUES
('1', '001', '01', '2005-10-21', 240),
('1', '001', '02', '2005-10-21', 48),
('1', '002', '03', '2005-10-22', 60),
('1', '004', '05', '2005-10-22', 4),
('2', '002', '03', '2005-10-22', 48),
('2', '002', '05', '2005-10-23', 2),
('2', '004', '01', '2005-10-23', 480),
('2', '004', '02', '2005-10-24', 72),
('3', '003', '03', '2005-10-24', 48),
('3', '003', '04', '2005-10-25', 20)

"""

crsr.execute(query_insert2)

- Luego de reparto entre parentesis, hago una tupla con el nombre de las columnas en el orden en el que voy a insertar los datos.

- El formato de datos de fecha es SIEMPRE = '2005-10-21' - Si no, Python no lo lee como fechas, por ejemplo, si se le pone barras, o el año al final, o el mes al final. 

- `parse_dates` = Si quiero que me convierta los datos de 'Fecha' a formato datetime hay que usar eso al leer el sql, ya que si no Pandas lee siempre de la manera mas sencilla, más cómoda para él, y seguro lo va a leer como un string. 
Esa sentencia se pone en `pd.read_sql('SELECT * FROM reparto', connection, parse_dates=['Fecha'])` > Acá le digo que a la columna 'Fecha' convierta los valores a tipo de datos datetime

> #### **SENTENCIA PARA BORRAR REGISTROS (fila, línea)**

- **SINTAXIS** 
**DELETE FROM table** <br>
**WHERE** 'search condition' >> **SIEMPRE poner WHERE ya que si no, SE BORRA TODA LA BASES DE DATOS, y NO se puede volver atrás!!!!** 

- Para borrar en el ejemplo, se utilizaría la sentencia: 
`query_delete = "DELETE FROM reparto WHERE CodB = '001'"`
`crsr.execute(query_delete)`
Se utiliza el `crsr.execute()` porque no se puede hacer adentro de `pd.read_sql()`, ya que eso solo lee archivos. 

In [22]:
# Insertamos datos en la BARES

bares = pd.DataFrame({
                    "CodB": ["001", "002", "003", "004"],
                    "Nombre": ["Stop", "Las Vegas", "Club Social", "Otra Ronda"],
                    "Cif": ["011111111X", "22222222Y", None, "33333333Z"],
                    "Localidad": ["Villa Botijo", "Villa Botijo", "Las Ranas", "La Esponja"] 
                    })

bares

bares.to_sql(name='bares', con=connection, if_exists="append", index=False)

4

In [23]:
pd.read_sql('SELECT * FROM bares', connection)

,CodB,Nombre,Cif,Localidad
0,001,Stop,011111111X,Villa Botijo
1,002,Las Vegas,22222222Y,Villa Botijo
2,003,Club Social,None,Las Ranas
3,004,Otra Ronda,33333333Z,La Esponja


In [24]:
# Insertamos datos en la EMPLEADOS

empleados = pd.DataFrame({
                    "CodE": ["1", "2", "3"],
                    "Nombre": ["Prudencio Caminero", "Vicente Merario", "Valentin Siempre"],
                    "Sueldo": ["120000", "110000", "100000"]
                    })

empleados

empleados.to_sql(name='empleados', con=connection, if_exists="append", index=False)

3

In [25]:
pd.read_sql('SELECT * FROM empleados', connection)

,CodE,Nombre,Sueldo
0,1,Prudencio Caminero,120000
1,2,Vicente Merario,110000
2,3,Valentin Siempre,100000


> #### **SINTAXIS PARA GUARDAR CAMBIOS REALIZADOS**

Lo siguiente se hace para guardar los cambios, `commit` por si quedo algo sin guardar, y `close` para cerrar la conexión a la base de datos, y que otros por ejemplo, se puedan conectar. 

In [ ]:
connection.commit()
connection.close()

> **Luego de cerrada la base de datos, para volver a conectarnos ejecutamos lo siguiente:** 


In [26]:
connection = sqlite3.connect("data/Bar_TB.db")

crsr = connection.cursor()

### RESPUESTAS

1. Obtener  el  nombre    de  los  empleados   que  hayan  repartido  al  bar  Stop  durante la semana del 17 al 23 de octubre de 2005

In [31]:
query1 = '''
SELECT DISTINCT
e.Nombre
FROM empleados e
INNER JOIN reparto r
ON e.CodE = r.CodE
INNER JOIN bares b
ON b.CodB = r.CodB
WHERE b.Nombre = 'Stop' AND r.Fecha BETWEEN "2005-10-17" AND "2005-10-23"
'''

sql_query(query1)

,Nombre
0,Prudencio Caminero
